In [1]:
import IPython.display as ipd

import numpy as np
import torch


# import stft_64_pad_0 as stft
import stft_64 as stft
from audio_processing import griffin_lim

from scipy.io.wavfile import read
import time
from reconstruct_functions import *
import random
import os

def set_seed(seed: int = 42) -> None:
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ["PYTHONHASHSEED"] = str(seed)
    print(f"Random seed set as {seed}")

set_seed(114514)

Random seed set as 114514


In [2]:
hop_length = 2
win_length = 8 ###at least 4*hop_length, even number
channels = win_length//2+1
full_length = 1000


stft_fn = stft.STFT(filter_length=win_length, hop_length=hop_length, win_length=win_length,
                    window=None)

In [3]:
def compute_reconstruction(input_signal):
    # shape: [1,:]
    magnitude, phase_origin = stft_fn.transform(audio_origin)
    magnitude    = magnitude[:,:,2:-2]
    ans = reconstruct_from_S_with_H_2(magnitude, stft_fn, hop_length=2, frame_number=None)
    if ans[0,0]*audio_origin[0,0]<0:
        ans[:,:] = -ans[:,:]
    return ans


In [4]:
test_time = 5
data_arr = []

for idx in range(test_time):
    audio_origin = (torch.rand((1,full_length), dtype=torch.float64)*2-1)
    ans = compute_reconstruction(audio_origin)
    data_arr.append(torch.mean(torch.abs(audio_origin[0,:] - ans[0,:])))
    print(torch.mean(torch.abs(audio_origin[0,:] - ans[0,:])))

print(np.mean(data_arr))
print(np.var(data_arr))

100%|██████████| 494/494 [00:14<00:00, 33.48it/s]


tensor(1.8016e-14, dtype=torch.float64)


100%|██████████| 494/494 [00:14<00:00, 33.93it/s]


tensor(4.3758e-14, dtype=torch.float64)


100%|██████████| 494/494 [00:14<00:00, 33.41it/s]


tensor(8.8737e-14, dtype=torch.float64)


100%|██████████| 494/494 [00:14<00:00, 33.15it/s]


tensor(1.5251e-14, dtype=torch.float64)


100%|██████████| 494/494 [00:14<00:00, 33.88it/s]

tensor(1.8453e-14, dtype=torch.float64)
3.6843030015720756e-14
7.79925926177456e-28
